In [1]:
import numpy as np
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torch.utils.data.sampler import SubsetRandomSampler
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import torchvision.models
from PIL import Image

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
 #Convolutional Neural Network Architecture
class GestureClassifier(nn.Module):
      def __init__(self):
          self.name = "GestureClassifier"
          super(GestureClassifier, self).__init__()
          self.conv1 = nn.Conv2d(3, 5, 5) #in_channels, out_chanels, kernel_size
          self.pool = nn.MaxPool2d(2, 2) #kernel_size, stride 
          self.conv2 = nn.Conv2d(5, 10, 5) #in_channels, out_chanels, kernel_size
          self.fc1 = nn.Linear(10*53*53, 30)
          self.fc2 = nn.Linear(30, 9)

      def forward(self, x):
          x = self.pool(F.relu(self.conv1(x)))
          x = self.pool(F.relu(self.conv2(x)))
          x = x.view(-1, 10*53*53)
          x = F.relu(self.fc1(x))
          x = self.fc2(x)
          return x